# 第5章 顧客の退会を予測する10本ノック (41~50)

In [1]:
# 下準備として，データのあるディレクトリに移動しておく
import os

DATA_ROOT_DIR = "./sample/5章/"
os.chdir(DATA_ROOT_DIR)

## ノック41 データを読み込んで利用データを成形しよう
今回は，データを読み込むだけでなく，データの成形も行う．

In [2]:
import pandas as pd
customer = pd.read_csv("customer_join.csv")
use_log_months = pd.read_csv("use_log_months.csv")

In [3]:
year_months = list(use_log_months["年月"].unique())
use_log = pd.DataFrame()

for i in range(1, len(year_months)):
    tmp = use_log_months.loc[use_log_months["年月"] == year_months[i]]
    tmp.rename(columns={"count":"count_0"}, inplace=True)
    tmp_before = use_log_months.loc[use_log_months["年月"] == year_months[i-1]]
    del tmp_before["年月"]
    tmp_before.rename(columns={"count":"count_1"}, inplace=True)
    tmp = pd.merge(tmp, tmp_before, on="customer_id", how="left")
    use_log = pd.concat([use_log, tmp], ignore_index=True)

use_log.head()

,年月,customer_id,count_0,count_1
0,201805,AS002855,5,4.0
1,201805,AS009373,4,3.0
2,201805,AS015233,7,NaN
3,201805,AS015315,3,6.0
4,201805,AS015739,5,7.0


## ノック42 退会前月の退会顧客データを作成しよう
このデータの場合，月末までに退会申請をすると，翌月に退会になる．そのため，退会の予測には，退会月の2ヵ月前のデータが必要になる．

In [4]:
from dateutil.relativedelta import relativedelta

exit_customer = customer.loc[customer["is_deleted"]==1]
exit_customer["exit_date"] = None
exit_customer["end_date"] = pd.to_datetime(exit_customer["end_date"])

for i in range(len(exit_customer)):
    exit_customer["exit_date"].iloc[i] = exit_customer["end_date"].iloc[i] - relativedelta(months=1)

# for文の中でうまくキャストされてない(バグかしら？)ので，手動でdatetime型にキャスト
exit_customer["exit_date"] = pd.to_datetime(exit_customer["exit_date"])

exit_customer["年月"] = exit_customer["exit_date"].dt.strftime("%Y%m")
use_log["年月"] = use_log["年月"].astype(str)
exit_use_log = pd.merge(use_log, exit_customer, on=["customer_id", "年月"], how="left")
print(len(use_log))
exit_use_log.head() 

33851


,年月,customer_id,count_0,count_1,name,class,gender,start_date,end_date,campaign_id,...,price,campaign_name,mean,median,max,min,routine_flg,calc_date,membership_period,exit_date
0,201805,AS002855,5,4.0,NaN,NaN,NaN,NaN,NaT,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaT
1,201805,AS009373,4,3.0,NaN,NaN,NaN,NaN,NaT,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaT
2,201805,AS015233,7,NaN,NaN,NaN,NaN,NaN,NaT,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaT
3,201805,AS015315,3,6.0,NaN,NaN,NaN,NaN,NaT,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaT
4,201805,AS015739,5,7.0,NaN,NaN,NaN,NaN,NaT,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaT


In [5]:
exit_use_log = exit_use_log.dropna(subset=["name"])
print(len(exit_use_log))
print(len(exit_use_log["customer_id"].unique()))
exit_use_log.head()

1104
1104


,年月,customer_id,count_0,count_1,name,class,gender,start_date,end_date,campaign_id,...,price,campaign_name,mean,median,max,min,routine_flg,calc_date,membership_period,exit_date
19,201805,AS055680,3,3.0,XXXXX,C01,M,2018-03-01,2018-06-30,CA1,...,10500.0,通常,3.000000,3.0,3.0,3.0,0.0,2018-06-30,3.0,2018-05-30
57,201805,AS169823,2,3.0,XX,C01,M,2017-11-01,2018-06-30,CA1,...,10500.0,通常,3.000000,3.0,4.0,2.0,1.0,2018-06-30,7.0,2018-05-30
110,201805,AS305860,5,3.0,XXXX,C01,M,2017-06-01,2018-06-30,CA1,...,10500.0,通常,3.333333,3.0,5.0,2.0,0.0,2018-06-30,12.0,2018-05-30
128,201805,AS363699,5,3.0,XXXXX,C01,M,2018-02-01,2018-06-30,CA1,...,10500.0,通常,3.333333,3.0,5.0,2.0,0.0,2018-06-30,4.0,2018-05-30
147,201805,AS417696,1,4.0,XX,C03,F,2017-09-01,2018-06-30,CA1,...,6000.0,通常,2.000000,1.0,4.0,1.0,0.0,2018-06-30,9.0,2018-05-30


## ノック43 継続顧客のデータを作成しよう
継続顧客のデータを作成し，アンダーサンプリングを行う．

In [6]:
conti_customer = customer.loc[customer["is_deleted"]==0]
conti_use_log = pd.merge(use_log, conti_customer, on=["customer_id"], how="left")
print(len(conti_use_log))
conti_use_log = conti_use_log.dropna(subset=["name"])
print(len(conti_use_log)) 

33851
27422


In [7]:
conti_use_log = conti_use_log.sample(frac=1).reset_index(drop=True)
conti_use_log = conti_use_log.drop_duplicates(subset="customer_id")
print(len(conti_use_log))
conti_use_log.head()

2842


,年月,customer_id,count_0,count_1,name,class,gender,start_date,end_date,campaign_id,...,class_name,price,campaign_name,mean,median,max,min,routine_flg,calc_date,membership_period
0,201808,AS124354,4,7.0,XXXX,C03,M,2015-09-01,NaN,CA1,...,ナイト,6000.0,通常,4.583333,4.0,7.0,3.0,1.0,2019-04-30,43.0
1,201902,IK313475,6,7.0,XXXX,C03,M,2015-07-01,NaN,CA1,...,ナイト,6000.0,通常,5.583333,5.5,7.0,3.0,1.0,2019-04-30,45.0
2,201808,OA660152,10,9.0,XXX,C03,M,2018-05-10,NaN,CA2,...,ナイト,6000.0,入会費半額,7.727273,9.0,10.0,4.0,1.0,2019-04-30,11.0
3,201903,AS408690,8,8.0,XXXX,C03,M,2018-09-01,NaN,CA1,...,ナイト,6000.0,通常,8.714286,9.0,11.0,5.0,1.0,2019-04-30,7.0
4,201812,PL779242,3,6.0,XXXX,C01,F,2015-06-01,NaN,CA1,...,オールタイム,10500.0,通常,4.500000,4.0,6.0,3.0,1.0,2019-04-30,46.0


In [8]:
predict_data = pd.concat([conti_use_log, exit_use_log], ignore_index=True)
print(len(predict_data))
predict_data.head()

3946


,年月,customer_id,count_0,count_1,name,class,gender,start_date,end_date,campaign_id,...,price,campaign_name,mean,median,max,min,routine_flg,calc_date,membership_period,exit_date
0,201808,AS124354,4,7.0,XXXX,C03,M,2015-09-01,NaN,CA1,...,6000.0,通常,4.583333,4.0,7.0,3.0,1.0,2019-04-30,43.0,NaT
1,201902,IK313475,6,7.0,XXXX,C03,M,2015-07-01,NaN,CA1,...,6000.0,通常,5.583333,5.5,7.0,3.0,1.0,2019-04-30,45.0,NaT
2,201808,OA660152,10,9.0,XXX,C03,M,2018-05-10,NaN,CA2,...,6000.0,入会費半額,7.727273,9.0,10.0,4.0,1.0,2019-04-30,11.0,NaT
3,201903,AS408690,8,8.0,XXXX,C03,M,2018-09-01,NaN,CA1,...,6000.0,通常,8.714286,9.0,11.0,5.0,1.0,2019-04-30,7.0,NaT
4,201812,PL779242,3,6.0,XXXX,C01,F,2015-06-01,NaN,CA1,...,10500.0,通常,4.500000,4.0,6.0,3.0,1.0,2019-04-30,46.0,NaT


## ノック44 予測する月の在籍期間を作成しよう
在籍期間を予想のための変数として用いる．

In [9]:
predict_data["now_date"] = pd.to_datetime(predict_data["年月"], format="%Y%m")
predict_data["start_date"] = pd.to_datetime(predict_data["start_date"])

deltas = [relativedelta(now, start) for now, start in zip(predict_data["now_date"], predict_data["start_date"])]
periods = [int(delta.years*12 + delta.months) for delta in deltas]

predict_data["period"] = periods
predict_data.head()

,年月,customer_id,count_0,count_1,name,class,gender,start_date,end_date,campaign_id,...,mean,median,max,min,routine_flg,calc_date,membership_period,exit_date,now_date,period
0,201808,AS124354,4,7.0,XXXX,C03,M,2015-09-01,NaN,CA1,...,4.583333,4.0,7.0,3.0,1.0,2019-04-30,43.0,NaT,2018-08-01,35
1,201902,IK313475,6,7.0,XXXX,C03,M,2015-07-01,NaN,CA1,...,5.583333,5.5,7.0,3.0,1.0,2019-04-30,45.0,NaT,2019-02-01,43
2,201808,OA660152,10,9.0,XXX,C03,M,2018-05-10,NaN,CA2,...,7.727273,9.0,10.0,4.0,1.0,2019-04-30,11.0,NaT,2018-08-01,2
3,201903,AS408690,8,8.0,XXXX,C03,M,2018-09-01,NaN,CA1,...,8.714286,9.0,11.0,5.0,1.0,2019-04-30,7.0,NaT,2019-03-01,6
4,201812,PL779242,3,6.0,XXXX,C01,F,2015-06-01,NaN,CA1,...,4.500000,4.0,6.0,3.0,1.0,2019-04-30,46.0,NaT,2018-12-01,42


## ノック45 欠損値を除去しよう
機械学習のための前処理として，欠損値を取り除く．

In [10]:
predict_data.isna().sum()

年月                      0
customer_id             0
count_0                 0
count_1               269
name                    0
class                   0
gender                  0
start_date              0
end_date             2842
campaign_id             0
is_deleted              0
class_name              0
price                   0
campaign_name           0
mean                    0
median                  0
max                     0
min                     0
routine_flg             0
calc_date               0
membership_period       0
exit_date            2842
now_date                0
period                  0
dtype: int64

In [11]:
predict_data = predict_data.dropna(subset=["count_1"])
predict_data.isna().sum()

年月                      0
customer_id             0
count_0                 0
count_1                 0
name                    0
class                   0
gender                  0
start_date              0
end_date             2625
campaign_id             0
is_deleted              0
class_name              0
price                   0
campaign_name           0
mean                    0
median                  0
max                     0
min                     0
routine_flg             0
calc_date               0
membership_period       0
exit_date            2625
now_date                0
period                  0
dtype: int64

##  ノック46 文字列型の変数を処理できるように整形しよう
機械学習に用いる，会員区分，性別などのカテゴリカル変数をダミー変数化する．

In [12]:
target_col = ["campaign_name", "class_name", "gender", "count_1", "routine_flg", "period", "is_deleted"]
predict_data = predict_data[target_col]
predict_data.head()

,campaign_name,class_name,gender,count_1,routine_flg,period,is_deleted
0,通常,ナイト,M,7.0,1.0,35,0.0
1,通常,ナイト,M,7.0,1.0,43,0.0
2,入会費半額,ナイト,M,9.0,1.0,2,0.0
3,通常,ナイト,M,8.0,1.0,6,0.0
4,通常,オールタイム,F,6.0,1.0,42,0.0


In [13]:
predict_data = pd.get_dummies(predict_data)
predict_data.head()

,count_1,routine_flg,period,is_deleted,campaign_name_入会費半額,campaign_name_入会費無料,campaign_name_通常,class_name_オールタイム,class_name_デイタイム,class_name_ナイト,gender_F,gender_M
0,7.0,1.0,35,0.0,0,0,1,0,0,1,0,1
1,7.0,1.0,43,0.0,0,0,1,0,0,1,0,1
2,9.0,1.0,2,0.0,1,0,0,0,0,1,0,1
3,8.0,1.0,6,0.0,0,0,1,0,0,1,0,1
4,6.0,1.0,42,0.0,0,0,1,1,0,0,1,0


In [14]:
del predict_data["campaign_name_通常"]
del predict_data["class_name_ナイト"]
del predict_data["gender_M"]
predict_data.head()

,count_1,routine_flg,period,is_deleted,campaign_name_入会費半額,campaign_name_入会費無料,class_name_オールタイム,class_name_デイタイム,gender_F
0,7.0,1.0,35,0.0,0,0,0,0,0
1,7.0,1.0,43,0.0,0,0,0,0,0
2,9.0,1.0,2,0.0,1,0,0,0,0
3,8.0,1.0,6,0.0,0,0,0,0,0
4,6.0,1.0,42,0.0,0,0,1,0,1


## ノック47 決定木を用いて退会予測モデルを作成してみよう
決定木で退会を予測する．

In [15]:
from sklearn.tree import DecisionTreeClassifier
import sklearn.model_selection

exit = predict_data.loc[predict_data["is_deleted"]==1]
conti = predict_data.loc[predict_data["is_deleted"]==0].sample(len(exit))

X = pd.concat([exit, conti], ignore_index=True)
y = X["is_deleted"]
del X["is_deleted"]

X_train, X_test, y_train, y_test = sklearn.model_selection.train_test_split(X, y)

model = DecisionTreeClassifier(random_state=0)
model.fit(X_train, y_train)
y_test_pred = model.predict(X_test)
print(y_test_pred)

[0. 0. 0. 1. 1. 0. 1. 0. 0. 1. 1. 0. 0. 1. 0. 1. 1. 0. 0. 0. 1. 1. 0. 1.
 0. 0. 0. 1. 0. 0. 0. 1. 0. 1. 0. 0. 0. 0. 1. 1. 0. 0. 1. 0. 1. 0. 1. 0.
 0. 1. 1. 0. 1. 0. 0. 1. 0. 1. 1. 0. 1. 0. 1. 0. 0. 1. 1. 1. 0. 0. 1. 1.
 1. 1. 0. 1. 1. 0. 0. 1. 1. 1. 0. 0. 0. 1. 0. 0. 0. 1. 0. 0. 0. 1. 0. 1.
 0. 1. 0. 1. 1. 1. 0. 1. 0. 1. 1. 1. 1. 0. 0. 0. 1. 0. 1. 0. 0. 0. 0. 1.
 0. 0. 0. 1. 1. 0. 0. 1. 0. 1. 1. 1. 1. 1. 1. 0. 1. 1. 0. 0. 1. 1. 0. 1.
 0. 0. 1. 1. 0. 1. 1. 0. 0. 0. 1. 0. 0. 0. 1. 1. 1. 1. 1. 1. 0. 0. 0. 1.
 0. 1. 1. 1. 0. 1. 0. 0. 1. 0. 0. 0. 0. 1. 0. 1. 1. 0. 1. 0. 0. 0. 0. 0.
 1. 0. 1. 0. 0. 1. 0. 0. 0. 1. 0. 1. 0. 0. 0. 1. 0. 1. 0. 1. 0. 1. 0. 1.
 0. 0. 1. 1. 0. 1. 0. 0. 0. 0. 0. 1. 0. 1. 0. 0. 1. 1. 0. 1. 1. 1. 0. 0.
 0. 0. 0. 0. 1. 1. 1. 1. 0. 1. 1. 0. 0. 0. 0. 0. 0. 0. 0. 0. 1. 0. 0. 1.
 0. 1. 0. 1. 1. 0. 1. 0. 0. 0. 0. 1. 0. 1. 1. 1. 0. 0. 0. 0. 0. 0. 1. 0.
 1. 1. 1. 1. 0. 0. 0. 1. 0. 0. 1. 0. 1. 1. 1. 1. 1. 0. 0. 1. 1. 1. 1. 0.
 0. 0. 0. 1. 0. 0. 1. 1. 0. 1. 1. 1. 1. 0. 1. 1. 0.

In [16]:
results_test = pd.DataFrame({"y_test":y_test, "y_pred":y_test_pred})
results_test.head()

,y_test,y_pred
1443,0.0,0.0
1197,0.0,0.0
1007,1.0,0.0
1883,0.0,1.0
725,1.0,1.0


## ノック48 予想モデルの評価を行い，モデルのチューニングをしてみよう
決定木の深さを調整する．


In [17]:
correct = len(results_test.loc[results_test["y_test"]==results_test["y_pred"]])
data_count = len(results_test)
score_test = correct / data_count
print(score_test)

0.8935361216730038


In [18]:
print(model.score(X_test, y_test))
print(model.score(X_train, y_train))

0.8935361216730038
0.9803548795944234


In [19]:
X = pd.concat([exit, conti], ignore_index=True)
y = X["is_deleted"]
del X["is_deleted"]
X_train, X_test, y_train, y_test = sklearn.model_selection.train_test_split(X, y)

model = DecisionTreeClassifier(random_state=0, max_depth=5)
model.fit(X_train, y_train)
print(model.score(X_test, y_test))
print(model.score(X_train, y_train))

0.903041825095057
0.9277566539923955
